In [173]:
import pandas as pd

In [175]:
df = pd.read_csv("Downloads/spotify_millsongdata.csv")

In [176]:
df.sample(5)

,artist,song,link,text
35627,Heart,Even It Up,/h/heart/even+it+up_20064707.html,"""I'm the one who can please you"" \r\nAin't th..."
13941,Nick Cave,Bring It On,/n/nick+cave/bring+it+on_20100059.html,This garden that I built for you \r\nThat you...
41034,Kylie Minogue,No Better,/k/kylie+minogue/no+better_20494655.html,Ah ah Ah ah Ah ah \r\nWonder in you're wilder...
44110,Misfits,Crimson Ghost,/m/misfits/crimson+ghost_20094044.html,Whoa \r\nI know your every move behind this f...
19642,Townes Van Zandt,"Fare Thee Well, Miss Carousel",/t/townes+van+zandt/fare+thee+well+miss+carous...,The drunken clown's still hanging round \r\nB...


In [179]:
df=df.sample(5000).drop('link',axis=1).reset_index(drop=True)

In [181]:
#df=df.sample(5000)

Text Preprocessing

In [184]:
df['text']=df['text'].str.lower().replace(r'^\w\s' , ' ').replace(r'\n',' ' , regex=True)

In [186]:
import nltk
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rajde\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [188]:
stemmer = PorterStemmer()

In [190]:
def token(txt):
    token = nltk.word_tokenize(txt)
    a=[stemmer.stem(w) for w in token]
    return " ".join(a)

In [192]:
token("you are beautiful, beauty")

'you are beauti , beauti'

In [194]:
df['text'].apply(lambda x:token(x))

0       ( instrument ) i 'm not look to chang your min...
1       mayb today mayb today you 'll find your love ....
2       i 'm stand on a hilltop a hundr mile from home...
3       hear - talk - ye i - love you set your heart s...
4       come ye broken heart come ye weari soul there ...
                              ...                        
4995    i set out on monday the night wa cold and vast...
4996    stare down from thi high window at the face in...
4997    lead me to the river of love pour a littl wate...
4998    in love and lone just to see you cri in love a...
4999    there is person live in my head she come to vi...
Name: text, Length: 5000, dtype: object

In [196]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [198]:
tfid = TfidfVectorizer(analyzer='word' , stop_words='english')

In [200]:
matrix = tfid.fit_transform(df['text'])

In [201]:
similar = cosine_similarity(matrix)

In [204]:
similar[0]

array([1.        , 0.08891484, 0.00863254, ..., 0.00585703, 0.03276752,
       0.01417562])

Recommender Function

In [217]:
def recommender(song_name):
    idx=df[df['song']==song_name].index[0]
    distance = sorted(list(enumerate(similar[idx])), reverse=True, key=lambda x:x[1])
    song = []
    for s_id in distance[1:5]:
        song.append(df.iloc[s_id[0]].song)
    return song

In [221]:
recommender("The Hammer Song")

['Hammer To Fall',
 'An Angel Came To Me',
 'Bling Blang',
 "Let's Get It Started"]

In [223]:
import pickle

In [225]:
pickle.dump(similar, open("similarity","wb"))

In [227]:
pickle.dump(df, open("df","wb"))